# Cost-Consequence Analysis (CCA) Example

This notebook demonstrates how to perform cost-consequence analysis, which presents costs and consequences separately without combining them into a single measure.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Define a simple CCA function
def run_cost_consequence_analysis(strategies, costs, consequences):
    """
    Perform basic cost-consequence analysis.
    
    Args:
        strategies: List of strategy names
        costs: List of costs for each strategy
        consequences: List of consequence dataframes for each strategy
        
    Returns:
        A combined dataframe with costs and consequences
    """
    result_data = []
    
    for i, strategy in enumerate(strategies):
        consequence_dict = consequences[i].to_dict('records')[0] if isinstance(cons equences[i], pd.DataFrame) else consequences[i]
        
        result_row = {
            'strategy': strategy,
            'total_cost': costs[i],
        }
        
        # Add consequences to the row
        result_row.update(consequence_dict)
        result_data.append(result_row)
    
    return pd.DataFrame(result_data)

In [ ]:
# Define example strategies and parameters
strategies = ['ECT', 'IV-KA', 'PO-KA']

# Define costs for each strategy
costs = [5000, 7500, 6000]  # Total cost per patient

# Define consequences (multiple metrics)
consequences = [
    {'qalys_gained': 0.6, 'response_rate': 0.65, 'adverse_events': 0.15, 'days_in_hospital': 12},
    {'qalys_gained': 0.8, 'response_rate': 0.82, 'adverse_events': 0.08, 'days_in_hospital': 5},
    {'qalys_gained': 0.7, 'response_rate': 0.75, 'adverse_events': 0.12, 'days_in_hospital': 8}
]

# Run CCA analysis
cca_results = run_cost_consequence_analysis(strategies, costs, consequences)
print(cca_results)

In [ ]:
# Visualize the CCA results
fig, ax = plt.subplots(2, 2, figsize=(15, 12))

# Costs comparison
ax[0, 0].bar(cca_results['strategy'], cca_results['total_cost'], color=['blue', 'green', 'orange'])
ax[0, 0].set_title('Total Costs by Strategy')
ax[0, 0].set_ylabel('Costs ($AUD)')
ax[0, 0].grid(True, alpha=0.3)

# QALYs gained
ax[0, 1].bar(cca_results['strategy'], cca_results['qalys_gained'], color=['blue', 'green', 'orange'])
ax[0, 1].set_title('QALYs Gained by Strategy')
ax[0, 1].set_ylabel('QALYs')
ax[0, 1].grid(True, alpha=0.3)

# Response rate
ax[1, 0].bar(cca_results['strategy'], cca_results['response_rate'], color=['blue', 'green', 'orange'])
ax[1, 0].set_title('Response Rate by Strategy')
ax[1, 0].set_ylabel('Response Rate')
ax[1, 0].set_ylim(0, 1)
ax[1, 0].grid(True, alpha=0.3)

# Adverse events
ax[1, 1].bar(cca_results['strategy'], cca_results['adverse_events'], color=['blue', 'green', 'orange'])
ax[1, 1].set_title('Adverse Events by Strategy')
ax[1, 1].set_ylabel('Adverse Events Rate')
ax[1, 1].set_ylim(0, max(cca_results['adverse_events'])*1.2)
ax[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create a CCA tornado plot to show trade-offs
plt.figure(figsize=(12, 8))

# Normalize values to compare across different scales
metrics = ['qalys_gained', 'response_rate', 'adverse_events', 'days_in_hospital']
metric_names = ['QALYs Gained', 'Response Rate', 'Adverse Events', 'Days in Hospital']

# Calculate relative values (compared to baseline - ECT)
baseline_values = cca_results.iloc[0][metrics]  # ECT values

# Calculate relative improvements
relative_values = []
for i, strategy in enumerate(cca_results['strategy']):
    if strategy == 'ECT':  # Skip baseline in relative comparison
        continue
    
    strategy_row = cca_results[cca_results['strategy'] == strategy].iloc[0]
    rel_data = {'strategy': strategy}
    
    for metric in metrics:
        if metric == 'adverse_events' or metric == 'days_in_hospital':
            # For negative metrics (lower is better), calculate improvement as baseline - strategy
            rel_data[metric] = baseline_values[metric] - strategy_row[metric]
        else:
            # For positive metrics (higher is better), calculate improvement as strategy - baseline
            rel_data[metric] = strategy_row[metric] - baseline_values[metric]
    
    relative_values.append(rel_data)

rel_df = pd.DataFrame(relative_values)

# Plot relative improvements
strategy_names = rel_df['strategy'].tolist()
for i, metric in enumerate(metrics):
    values = rel_df[metric].tolist()
    y_pos = np.arange(len(strategy_names)) + i * len(strategy_names) * 0.2
    
    colors = ['green' if val > 0 else 'red' for val in values]
    bars = plt.barh(y_pos, values, height=0.8, label=metric_names[i], color=colors, alpha=0.7)

# Add baseline (dashed line at 0)
plt.axvline(x=0, color='black', linestyle='--', alpha=0.7)

plt.xlabel('Relative Improvement vs ECT')
plt.title('Cost-Consequence Analysis: Relative Improvements vs ECT')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Create a decision matrix to help with multi-criteria decision making
weights = {
    'qalys_gained': 0.35,
    'response_rate': 0.25,
    'adverse_events': -0.25,  # Negative weight since lower is better
    'days_in_hospital': -0.15  # Negative weight since lower is better
}

# Normalize values to 0-1 scale
normalized_results = cca_results.copy()
for metric in ['qalys_gained', 'response_rate']:
    min_val = cca_results[metric].min()
    max_val = cca_results[metric].max()
    if max_val != min_val:
        normalized_results[f'{metric}_norm'] = (cca_results[metric] - min_val) / (max_val - min_val)
    else:
        normalized_results[f'{metric}_norm'] = 0.5  # If all values are the same

for metric in ['adverse_events', 'days_in_hospital']:
    min_val = cca_results[metric].min()
    max_val = cca_results[metric].max()
    if max_val != min_val:
        # For negative metrics, reverse the scale (lower is better)
        normalized_results[f'{metric}_norm'] = (max_val - cca_results[metric]) / (max_val - min_val)
    else:
        normalized_results[f'{metric}_norm'] = 0.5

# Calculate decision scores
for i, row in normalized_results.iterrows():
    score = 0
    for metric, weight in weights.items():
        score += row[f'{metric}_norm'] * weight
    normalized_results.at[i, 'decision_score'] = score

print("Decision Matrix with Normalized Values and Scores:")
score_cols = ['strategy'] + [f'{m}_norm' for m in weights.keys()] + ['decision_score']
print(normalized_results[score_cols])

# Plot decision scores
plt.figure(figsize=(10, 6))
plt.bar(normalized_results['strategy'], normalized_results['decision_score'], 
        color=['blue', 'green', 'orange'])
plt.ylabel('Decision Score')
plt.title('Multi-Criteria Decision Score Based on CCA Results')
plt.grid(True, alpha=0.3)
for i, v in enumerate(normalized_results['decision_score']):
    plt.text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
plt.show()

In [ ]:
# Create a cost-consequence plane visualization
plt.figure(figsize=(12, 8))

# Select two key consequences to plot against cost
# Using QALYs gained vs cost
plt.scatter(cca_results['qalys_gained'], cca_results['total_cost'], 
           s=200, c=range(len(cca_results)), cmap='viridis', alpha=0.7)

# Add strategy labels
for i, strategy in enumerate(cca_results['strategy']):
    plt.annotate(strategy, 
                (cca_results['qalys_gained'].iloc[i], cca_results['total_cost'].iloc[i]),
                xytext=(5, 5), textcoords='offset points')

plt.xlabel('Health Consequences (QALYs Gained)')
plt.ylabel('Total Costs ($AUD)')
plt.title('Cost-Consequence Plane')
plt.grid(True, alpha=0.3)

# Add cost-effectiveness lines for reference
wtp = 50000
ref_qalys, ref_costs = cca_results['qalys_gained'].iloc[0], cca_results['total_cost'].iloc[0]
x_line = np.linspace(min(cca_results['qalys_gained']), max(cca_results['qalys_gained']), 100)
y_line = ref_costs + wtp * (x_line - ref_qalys)
plt.plot(x_line, y_line, 'r--', alpha=0.5, label=f'WTP = ${wtp:,}/QALY')
plt.legend()

plt.tight_layout()
plt.show()

## Next Steps

1. Integrate with actual cost and outcome data
2. Perform sensitivity analysis on consequence measures
3. Consider different perspectives (patient, health system, societal)
4. Combine with MCDA for multi-criteria decision analysis